### Imports

In [4]:
import pandas as pd
import numpy as np

### Reading In Different CSVs

In [6]:
def getstandings(year): #function to pull features to predict wins from the years 2009-2019
    hittingdata = pd.read_csv(f'data/{year}.csv') #read in hitting statistics
    Teamdic = {'HOU':'Astros', 'LAD':'Dodgers', 'CHC':'Cubs', 'ARI':'Diamondbacks', 'MIL':'Brewers', 'TBR':'Rays',
        'ATL':'Braves', 'BOS':'Red Sox', 'CLE': 'Indians','NYY':'Yankees','OAK': 'Athletics','STL':'Cardinals', 
        'SFG':'Giants','PIT':'Pirates', 'WSN': 'Nationals','NYM': 'Mets','SEA': 'Mariners','PHI':'Phillies','LAA':'Angels',
        'COL':'Rockies', 'SDP':'Padres', 'MIN':'Twins', 'DET':'Tigers','CIN': 'Reds','MIA':'Marlins','FLA':'Marlins','CHW':'White Sox',
        'TOR': 'Blue Jays', 'TEX':'Rangers', 'KCR' : 'Royals', 'BAL':'Orioles'} #Dictionary used to change all team ids to team names  
    hittingdata.Team.replace(Teamdic, inplace=True) #change all team ids to team names  
    hittingdata.sort_values(by = 'Team', inplace=True) #alphabetize teams 
    hittingdata.set_index('Team', inplace= True) #alphabetize teams
    pitchingdata = pd.read_csv(f'data/{year}P.csv')  #read in pitching statistics
    pitchingdata.drop('W', axis = 1, inplace=True) #drop wins.. we will bring in our own wins
    pitchingdata.drop('L', axis = 1, inplace=True) #correlate way too much to wins... drop them
    pitchingdata.Team.replace(Teamdic, inplace=True)#change all team ids to team names  
    pitchingdata.sort_values(by = 'Team', inplace=True)  #alphabetize teams 
    pitchingdata.set_index('Team', inplace= True)#make 'team' the index to merge on later
    winsdata = pd.read_csv(f'data/{year-1}P.csv').sort_values(by = 'Team') #read in data from the year before to try and predict the future wins
    winsdata.Team.replace(Teamdic, inplace=True)  #change all team ids to team names  
    winsdata.sort_values(by = 'Team', inplace=True)#alphabetize teams
    winsdata.set_index('Team', inplace= True)#alphabetize teams
    wins = winsdata['W'] #pull wins from the year before
    pitchingdata['W'] = wins #add wins from previous year to pitchind data
    pitchingdata.columns = pitchingdata.columns + './P' #add a ./P to distinguish hitting and pitching stats
    pitchingdata.rename(columns={"Team./P": "Team"}, inplace = True) #rename team./p to team in order to merge on team
    df_both = pd.merge(pitchingdata, hittingdata, on='Team', how = 'outer') #merge pitching and hitting stats
    df_both.dropna(axis = 1, inplace=True)# remove nans
    return df_both 

In [7]:
finallist = [] #empty list for later 
for year in range(2009, 2019): #for loop that will take the csvs from year 2009 - 2019
    finallist.append(getstandings(year)) #add these dataframes to that original empty list

In [8]:
finaldf = pd.concat(finallist, axis=0, join='inner').sort_index() #turn that list into a dataframe

In [9]:
finaldf.shape #make sure we have all 300 columns of data with the 527 features

(300, 527)

In [10]:
finaldf#look at dataframe to make sure it looks right

,SV./P,G./P,GS./P,IP./P,K/9./P,BB/9./P,HR/9./P,BABIP./P,LOB%./P,GB%./P,...,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Barrels,HardHit,Events
Team,,,,,,,,,,,,,,,,,,,,,
Angels,51,596,162,1445.0,6.61,3.26,1.12,0.301,72.4%,41.7%,...,98,96,102,103,95,100,102,0,0,0
Angels,41,658,162,1457.2,7.41,3.29,1.03,0.300,72.3%,42.1%,...,105,93,106,103,91,102,101,0,0,0
Angels,43,705,162,1440.2,8.20,2.94,1.40,0.289,74.9%,41.6%,...,91,105,99,94,106,102,94,210,1364,4282
Angels,29,689,162,1421.1,7.19,3.15,1.32,0.301,75.0%,40.8%,...,78,95,105,102,101,102,96,220,1533,4528
Angels,46,705,162,1482.2,8.15,3.06,0.76,0.285,73.3%,41.7%,...,109,101,98,101,97,97,107,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yankees,47,627,162,1458.1,7.54,3.13,0.94,0.297,75.0%,44.2%,...,143,105,96,97,100,96,109,0,0,0
Yankees,39,592,162,1442.1,7.20,3.37,1.12,0.281,74.0%,43.9%,...,130,103,97,99,107,93,108,0,0,0
Yankees,51,623,162,1450.0,7.82,3.56,1.12,0.290,71.9%,42.3%,...,133,100,100,101,108,97,103,0,0,0


In [11]:
finaldf.isnull().sum().sum() #check nulls just in case

0

In [343]:
finaldf.to_csv('finaldata.csv',index=False) #Save this datframe to a csv

------------------------------

In [12]:
def target(year): #function to pull target 'wins' from the years 2009-2019 but this time have the wins from the year after 
    hittingdata = pd.read_csv(f'data/{year}.csv')
    Teamdic = {'HOU':'Astros', 'LAD':'Dodgers', 'CHC':'Cubs', 'ARI':'Diamondbacks', 'MIL':'Brewers', 'TBR':'Rays',
        'ATL':'Braves', 'BOS':'Red Sox', 'CLE': 'Indians','NYY':'Yankees','OAK': 'Athletics','STL':'Cardinals', 
        'SFG':'Giants','PIT':'Pirates', 'WSN': 'Nationals','NYM': 'Mets','SEA': 'Mariners','PHI':'Phillies','LAA':'Angels',
        'COL':'Rockies', 'SDP':'Padres', 'MIN':'Twins', 'DET':'Tigers','CIN': 'Reds','MIA':'Marlins','FLA':'Marlins','CHW':'White Sox',
        'TOR': 'Blue Jays', 'TEX':'Rangers', 'KCR' : 'Royals', 'BAL':'Orioles'}  
    hittingdata.Team.replace(Teamdic, inplace=True)
    hittingdata.sort_values(by = 'Team', inplace=True)
    hittingdata.set_index('Team', inplace= True)
    pitchingdata = pd.read_csv(f'data/{year}P.csv')
    pitchingdata.drop('W', axis = 1, inplace=True)
    pitchingdata.drop('L', axis = 1, inplace=True)
    pitchingdata.Team.replace(Teamdic, inplace=True)
    pitchingdata.sort_values(by = 'Team', inplace=True)
    pitchingdata.set_index('Team', inplace= True)
    winsdata = pd.read_csv(f'data/{year+1}P.csv').sort_values(by = 'Team')
    winsdata.Team.replace(Teamdic, inplace=True)
    winsdata.sort_values(by = 'Team', inplace=True)
    winsdata.set_index('Team', inplace= True)
    wins = winsdata['W']
    pitchingdata['W'] = wins
    pitchingdata.columns = pitchingdata.columns + './P'
    pitchingdata.rename(columns={"Team./P": "Team"}, inplace = True)
    df_both = pd.merge(pitchingdata, hittingdata, on='Team', how = 'outer')
    df_both.dropna(axis = 1, inplace=True)# remove nans
    return df_both

In [13]:
targetlist = []
for year in range(2009, 2019): 
    targetlist.append(target(year)) 

In [14]:
targetdf = pd.concat(targetlist, axis=0, join='inner').sort_index()

In [15]:
ytargets = targetdf['W./P']

In [16]:
ytargets.to_csv('ytargets.csv',index=False) #Save this datframe to a csv

-----

------------------------------------------------------------------------------------------